In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("/content/cleaned_headlines.csv")

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [4]:
data.head()

,Unnamed: 0,date,company,cleaned_headline
0,0,2025-05-29,GOOGL,bill nygrens strategic moves constellation bra...
1,1,2025-05-29,TSLA,market today nvidia nvda leads ai surge amid s...
2,2,2025-05-29,AAPL,asx market open trump tariff appeal ruling bum...
3,6,2025-05-29,AAPL,sonder holdings inc receives notification of d...
4,7,2025-05-29,MSFT,netapp reports record results for fourth quart...


In [5]:
headlines = data['cleaned_headline'].tolist()

In [6]:
import torch

inputs = tokenizer(headlines, padding=True, truncation=True, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)

In [7]:
import torch.nn.functional as F
predictions = F.softmax(outputs.logits, dim=-1)

In [8]:
labels = ['positive', 'negative', 'neutral']
predicted_classes = predictions.argmax(axis=1)
data['sentiment'] = [labels[i] for i in predicted_classes]
data['sentiment_score'] = predictions.max(axis=1).values

In [9]:
data

,Unnamed: 0,date,company,cleaned_headline,sentiment,sentiment_score
0,0,2025-05-29,GOOGL,bill nygrens strategic moves constellation bra...,neutral,0.784569
1,1,2025-05-29,TSLA,market today nvidia nvda leads ai surge amid s...,positive,0.926326
2,2,2025-05-29,AAPL,asx market open trump tariff appeal ruling bum...,neutral,0.644258
3,6,2025-05-29,AAPL,sonder holdings inc receives notification of d...,negative,0.947585
4,7,2025-05-29,MSFT,netapp reports record results for fourth quart...,positive,0.922512
...,...,...,...,...,...,...
321,389,2025-05-27,TSLA,tesla struggles in europe as sales plunge 49 y...,negative,0.966155
322,390,2025-05-27,GOOGL,youtube ceo neal mohan says the streaming gian...,negative,0.822908
323,391,2025-05-27,AAPL,stock market today markets rally as trump dela...,negative,0.774963
324,392,2025-05-27,MSFT,stock market today markets rally as trump dela...,negative,0.774963


In [10]:
data.drop('Unnamed: 0', axis =1, inplace = True)

In [11]:
data.to_csv("final_clean_headlines.csv")